### 동적 스크레이핑으로 모든 도서의 링크만 가져오기
- 유효하지 않은 3개의 링크는 추후 json 파일 편집으로 제거함 ( 링크 대신 javascript:void(0); 이 기록된 경우)

In [21]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

import time
import json

# WebDriver 설정
driver = webdriver.Chrome()
driver.get('https://www.yes24.com/Product/Category/Display/001001047001')
time.sleep(2)  # 페이지 로드 대기

# 판매량 순 버튼 클릭
sales_rank_button = driver.find_element(By.CSS_SELECTOR, "#categoryProductContentsWrap > div.cateGoodsSecArea > div > span.baseFilter > a:nth-child(2)")
sales_rank_button.click()
time.sleep(2)  # 변경 적용 대기

# 책 보기 개수 설정 (120개)
page_size_dropdown = Select(driver.find_element(By.CSS_SELECTOR, "#pg_size"))
page_size_dropdown.select_by_value("120")  # 120개 선택
time.sleep(2)  # 변경 적용 대기


# 스크롤 다운 함수
def scroll_down(driver, scroll_count):
    for _ in range(scroll_count):
        driver.execute_script("window.scrollBy(0, 1000);")  # 스크롤 내리기
        time.sleep(0.5)  # 대기

# 메인 페이지 스크롤
main_scroll_cnts = 9

# 페이지 번호 클릭 -> 이동
Total_pages_num = 25
book_links = []

for page_num in range(1, Total_pages_num+1):
    try:
        scroll_down(driver, main_scroll_cnts)

        # ✅ 모든 페이지에서 책 링크 수집
        books = driver.find_elements(By.CSS_SELECTOR, 'a.gd_name')
        for book in books:
            book_links.append(book.get_attribute('href'))
        print(f"페이지 {page_num}에서 수집한 링크 개수: {len(book_links)}")

        # ✅ 1페이지 (현재 페이지라 클릭 X)
        if page_num == 1:
            print(f"현재 페이지: {page_num}")

        # ✅ 2~9페이지 (nth-child로 클릭)
        elif 2 <= page_num <= 9:
            if page_num == 2:
                    page_link_selector = '#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong'  # 현재 페이지
            else:
                    nth_child_index = page_num  # 현재 페이지 기준으로 nth-child 결정
                    page_link_selector = f'#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child({nth_child_index})'
                
            page_link = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, page_link_selector))
                )
            page_link.click()
            print(f"이동한 페이지: {page_num}")
            time.sleep(1)

        # ✅ 10페이지 (next 버튼 클릭)
        elif page_num == 10:

            next_button = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, '#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next'))
                )
            next_button.click()
            print(f"페이지 {page_num}에서 'next button' 클릭")
            time.sleep(1)

        # ✅ 11~19페이지 (nth-child 조정)
        elif 11 <= page_num <= 19:
            if page_num == 11:
                page_link_selector = '#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong'  # 현재 페이지
            else:
                nth_child_index = (page_num - 10) + 3  # 11페이지는 strong, 이후 nth-child 증가
                page_link_selector = f'#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child({nth_child_index})'
                
            page_link = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, page_link_selector))
            )
            page_link.click()
            print(f"이동한 페이지: {page_num}")
            time.sleep(1)


        # ✅ 20페이지 (next 버튼 클릭)
        elif page_num == 20:
            next_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next'))
                )
            next_button.click()
            print(f"페이지 {page_num}에서 'next button' 클릭")
            time.sleep(1)


        # ✅ 21~25페이지 (nth-child 조정)
        elif 21 <= page_num <= 25:
            if page_num == 21:
                page_link_selector = '#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong'
            else:
                nth_child_index = (page_num - 20) + 3
                page_link_selector = f'#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child({nth_child_index})'

            page_link = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, page_link_selector))
                )
            page_link.click()
            print(f"이동한 페이지: {page_num}")
            time.sleep(2)


    except Exception as e:
        print(f"오류 발생 (페이지 번호 {page_num}): {e}")
        continue

# ✅ JSON 파일로 저장
with open("new_book_links2.json", "w", encoding="utf-8") as json_file:
    json.dump(book_links, json_file, ensure_ascii=False, indent=4)

print("크롤링 완료 및 JSON 파일 저장 완료!")

# ✅ Selenium 종료
driver.quit()

# ## 페이지 html 리스트#1_10
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong    #1
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(2) #2
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(10) #3
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next #4
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.end #5

# SY 11~20
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong #11
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(4) #12
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(5) #13
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(6) #14
##categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(12) #20 번 페이지


# next button
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next

페이지 1에서 수집한 링크 개수: 120
현재 페이지: 1
페이지 2에서 수집한 링크 개수: 240
이동한 페이지: 2
페이지 3에서 수집한 링크 개수: 360
이동한 페이지: 3
페이지 4에서 수집한 링크 개수: 480
이동한 페이지: 4
페이지 5에서 수집한 링크 개수: 600
이동한 페이지: 5
페이지 6에서 수집한 링크 개수: 720
이동한 페이지: 6
페이지 7에서 수집한 링크 개수: 840
이동한 페이지: 7
페이지 8에서 수집한 링크 개수: 960
이동한 페이지: 8
페이지 9에서 수집한 링크 개수: 1080
이동한 페이지: 9
페이지 10에서 수집한 링크 개수: 1200
페이지 10에서 'next button' 클릭
페이지 11에서 수집한 링크 개수: 1320
이동한 페이지: 11
페이지 12에서 수집한 링크 개수: 1440
이동한 페이지: 12
페이지 13에서 수집한 링크 개수: 1560
이동한 페이지: 13
페이지 14에서 수집한 링크 개수: 1680
이동한 페이지: 14
페이지 15에서 수집한 링크 개수: 1800
이동한 페이지: 15
페이지 16에서 수집한 링크 개수: 1920
이동한 페이지: 16
페이지 17에서 수집한 링크 개수: 2040
이동한 페이지: 17
페이지 18에서 수집한 링크 개수: 2160
이동한 페이지: 18
페이지 19에서 수집한 링크 개수: 2280
이동한 페이지: 19
페이지 20에서 수집한 링크 개수: 2400
페이지 20에서 'next button' 클릭
페이지 21에서 수집한 링크 개수: 2520
이동한 페이지: 21
페이지 22에서 수집한 링크 개수: 2640
이동한 페이지: 22
페이지 23에서 수집한 링크 개수: 2760
이동한 페이지: 23
페이지 24에서 수집한 링크 개수: 2880
이동한 페이지: 24
페이지 25에서 수집한 링크 개수: 3000
이동한 페이지: 25
크롤링 완료 및 JSON 파일 저장 완료!


In [22]:
# 페이지 25 / 수집한 링크 개수: 3000
len(book_links)

3000

# 정적 웹스크레이핑

In [23]:
import json
import re
from datetime import datetime
import requests
from bs4 import BeautifulSoup


with open('new_book_links2.json', 'r', encoding='utf-8') as file:
    all_book_links = json.load(file)

all_book_data = []
invalid_url_cnt = 0
invalid_url_idx = []
img_url_error_count = 0
img_url_error_idx = []
isbn13_error_count = 0
isbn13_error_idx = []
date_error_count = 0
date_error_idx = []

# test_link = ['https://www.yes24.com/Product/Goods/1392637']
# for idx, link in enumerate(test_link):



for idx, link in enumerate(all_book_links):

    if not link.startswith('http'):
        print(f"Invalid URL skipped : {link}")
        invalid_url_cnt += 1
        invalid_url_idx.append(idx)
        continue

    response = requests.get(link)
    
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    ## 줄바꿈으로 부제도 표현됨.
    titles = soup.select('h2.gd_name')
    title_text = '\n'.join(set([title.get_text(strip=True) for title in titles]))

    # 도서별 책소개의 css는 총 2종류이다
    descriptions = soup.select('#infoset_introduce > div.infoSetCont_wrap > div.infoWrap_txt > div')
    if len(descriptions) == 0:
        descriptions = soup.select('#infoset_introduce > div.infoSetCont_wrap > table > tbody > tr > td > div > div')
    descriptions_text = '\n'.join([desc.get_text(strip=True) for desc in descriptions])

    key_sentences = soup.select('#infoset_inBook > div.infoSetCont_wrap > div.infoWrap_txt > div')
    key_sentences_text = '\n'.join([key_sentence.get_text(strip=True) for key_sentence in key_sentences])

    if descriptions_text and key_sentences_text:
        authors = soup.select('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth')
        authors_text = ', '.join([author.get_text(strip=True, separator=' ') for author in authors])        

        publishers = soup.select('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_pub > a')
        publishers_text = ', '.join([publisher.get_text(strip=True) for publisher in publishers])
        
        isbn13_element = soup.select('#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr')
        if len(isbn13_element) > 1:
            isbn13 = isbn13_element[-2]
            if isbn13:
                isbn13_text = isbn13.get_text(strip=True)
                isbn13_num_only = re.sub(r'[^0-9]', '', isbn13_text)
                try:
                    isbn13_int = int(isbn13_num_only)
                except ValueError:
                    print(f"Invalid ISBN data: {isbn13_num_only}")
                    isbn13_int = isbn13_text
                    isbn13_error_count += 1
                    isbn13_error_idx.append(idx)
            else:
                isbn13 = None
        else:
            isbn13_int = None
            
        img_url = soup.select_one('img.gImg')['src']
        if img_url is None:
            img_url_error_count += 1
            img_url_error_idx.append(idx)

        publication_date = soup.select_one('span.gd_date')
        if publication_date is not None:
            publication_date_text = publication_date.text
            parsed_date = datetime.strptime(publication_date_text, "%Y년 %m월 %d일")
            formatted_date = parsed_date.strftime("%Y-%m-%d")
        else:
            print(f"Invalid Publication_date: {idx}th book")
            formatted_date = None
            date_error_count += 1
            date_error_idx.append(idx)


        all_book_data.append({
                'title': title_text,
                'description': descriptions_text,
                'key_sentences': key_sentences_text,
                'author': authors_text,
                'publisher': publishers_text,
                'isbn13': isbn13_int,
                'img_url': img_url,
                'publication_date': formatted_date
            })
        
    print(f'{idx}-th completed')
    

with open('new_book_data_ver.json', 'w', encoding='utf-8') as f:
    json.dump(all_book_data, f, ensure_ascii=False, indent=4)

0-th completed
1-th completed
2-th completed
3-th completed
4-th completed
5-th completed
6-th completed
7-th completed
8-th completed
9-th completed
10-th completed
11-th completed
12-th completed
13-th completed
14-th completed
15-th completed
16-th completed
17-th completed
18-th completed
19-th completed
20-th completed
21-th completed
22-th completed
23-th completed
24-th completed
25-th completed
26-th completed
27-th completed
28-th completed
29-th completed
30-th completed
31-th completed
32-th completed
33-th completed
34-th completed
35-th completed
36-th completed
37-th completed
38-th completed
39-th completed
40-th completed
41-th completed
42-th completed
43-th completed
44-th completed
45-th completed
46-th completed
47-th completed
48-th completed
49-th completed
50-th completed
51-th completed
52-th completed
53-th completed
54-th completed
55-th completed
56-th completed
57-th completed
58-th completed
59-th completed
60-th completed
61-th completed
62-th completed
63

In [24]:
print(len(all_book_data))

print(f"{invalid_url_cnt = }")
print(f'{invalid_url_idx = }')
print(f'{img_url_error_count = }')
print(f"{img_url_error_idx = }")
print(f"{isbn13_error_count = }")
print(f"{isbn13_error_idx = }")
print(f"{date_error_count = }")
print(f"{date_error_idx = }")

2267
invalid_url_cnt = 0
invalid_url_idx = []
img_url_error_count = 0
img_url_error_idx = []
isbn13_error_count = 0
isbn13_error_idx = []
date_error_count = 0
date_error_idx = []


In [26]:
import json

# ✅ JSON 파일 로드
with open("new_book_data_ver.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ 중복 제거를 위한 딕셔너리 활용 (ISBN 기준으로 유일한 책만 저장)
unique_books = {}
for book in data:
    isbn = book["isbn13"]
    if isbn not in unique_books:
        unique_books[isbn] = book  # 첫 번째 항목만 저장

# ✅ 중복 제거된 데이터 리스트 생성
unique_data = list(unique_books.values())

# ✅ 중복된 ISBN 개수 출력
original_count = len(data)
new_count = len(unique_data)
print(f"중복 제거 전: {original_count}개")
print(f"중복 제거 후: {new_count}개")
print(f"제거된 중복 개수: {original_count - new_count}개")

# ✅ 새로운 JSON 파일로 저장
with open("book_data_unique.json", "w", encoding="utf-8") as f:
    json.dump(unique_data, f, ensure_ascii=False, indent=4)

print("✅ 중복 제거 완료! 'book_data_unique.json'에 저장됨.")


중복 제거 전: 2267개
중복 제거 후: 1883개
제거된 중복 개수: 384개
✅ 중복 제거 완료! 'book_data_unique.json'에 저장됨.


In [27]:
import json

# ✅ JSON 파일 로드
with open("book_data_unique.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ 필터링된 데이터를 저장할 리스트
filtered_data = []

for book in data:
    # ✅ 기존의 `isbn13` 키를 `isbn`으로 변경
    if "isbn13" in book:
        original_isbn = str(book["isbn13"])  # ISBN을 문자열로 변환
        new_isbn = original_isbn[2:]  # 앞의 "13" 제거

        # ✅ 필터링: '97' 또는 '98'로 시작하는 ISBN만 유지
        if new_isbn.startswith(("97", "98")):
            book["isbn"] = new_isbn  # 새로운 ISBN 저장
            del book["isbn13"]  # 기존 `isbn13` 키 삭제
            filtered_data.append(book)  # 유효한 데이터만 저장

# ✅ 결과 출력
original_count = len(data)
filtered_count = len(filtered_data)
print(f"📌 원본 데이터 개수: {original_count}")
print(f"✅ 필터링 후 데이터 개수: {filtered_count}")
print(f"🗑️ 제거된 데이터 개수: {original_count - filtered_count}")

# ✅ 새로운 JSON 파일로 저장
with open("filtered_book_unique.json", "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)

print("✅ 필터링 완료! 'filtered_book_unique.json'에 저장됨.")

📌 원본 데이터 개수: 1883
✅ 필터링 후 데이터 개수: 1781
🗑️ 제거된 데이터 개수: 102
✅ 필터링 완료! 'filtered_book_unique.json'에 저장됨.
